<a href="https://colab.research.google.com/github/Sidra2499/Sale_Data_Analysis/blob/main/SalesData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local").appName("GoogleColabSpark").getOrCreate()

In [2]:
import matplotlib.pyplot as plt


In [3]:
from google.colab import files
unloaded = files.upload()



Saving sales.csv to sales.csv


In [4]:
print(spark.version)

3.5.4


In [5]:
sales_data = spark.read.csv('sales.csv', header= True, inferSchema= True )
sales_data.show()


+-------+------+-----------+-------------+------+------------+----------------+----------+--------+-----+-----------+-------------+
|sale_id|branch|       city|customer_type|gender|product_name|product_category|unit_price|quantity|  tax|total_price|reward_points|
+-------+------+-----------+-------------+------+------------+----------------+----------+--------+-----+-----------+-------------+
|      1|     A|   New York|       Member|  Male|     Shampoo|   Personal Care|       5.5|       3| 1.16|      17.66|            1|
|      2|     B|Los Angeles|       Normal|Female|    Notebook|      Stationery|      2.75|      10| 1.93|      29.43|            0|
|      3|     A|   New York|       Member|Female|       Apple|          Fruits|       1.2|      15| 1.26|      19.26|            1|
|      4|     A|    Chicago|       Normal|  Male|   Detergent|       Household|       7.8|       5| 2.73|      41.73|            0|
|      5|     B|Los Angeles|       Member|Female|Orange Juice|       Beverag

In [6]:
sales_data.printSchema()

root
 |-- sale_id: integer (nullable = true)
 |-- branch: string (nullable = true)
 |-- city: string (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- tax: double (nullable = true)
 |-- total_price: double (nullable = true)
 |-- reward_points: integer (nullable = true)



In [7]:
sales_data.describe().show()

+-------+-----------------+------+--------+-------------+------+------------+----------------+------------------+------------------+-----------------+------------------+-----------------+
|summary|          sale_id|branch|    city|customer_type|gender|product_name|product_category|        unit_price|          quantity|              tax|       total_price|    reward_points|
+-------+-----------------+------+--------+-------------+------+------------+----------------+------------------+------------------+-----------------+------------------+-----------------+
|  count|             1000|  1000|    1000|         1000|  1000|        1000|            1000|              1000|              1000|             1000|              1000|             1000|
|   mean|            500.5|  NULL|    NULL|         NULL|  NULL|        NULL|            NULL|10.836109999999996|            10.337|7.758010000000006|118.58390000000006|            6.057|
| stddev|288.8194360957494|  NULL|    NULL|         NULL|  N

In [8]:
print("Number of Rows", sales_data.count())
print("Number of Columns", len(sales_data.columns))

Number of Rows 1000
Number of Columns 12


In PySpark, you can't use isnull() like in pandas. Instead, you have to calculate how many missing (null) values are present in each column.

In [9]:
from pyspark.sql.functions import col, sum as _sum


In [10]:
# Count null values in each column
missing_data = sales_data.select(
    *[_sum(col(column).isNull().cast("int")).alias(column) for column in sales_data.columns]
)

# Show the missing data summary
missing_data.show()


+-------+------+----+-------------+------+------------+----------------+----------+--------+---+-----------+-------------+
|sale_id|branch|city|customer_type|gender|product_name|product_category|unit_price|quantity|tax|total_price|reward_points|
+-------+------+----+-------------+------+------------+----------------+----------+--------+---+-----------+-------------+
|      0|     0|   0|            0|     0|           0|               0|         0|       0|  0|          0|            0|
+-------+------+----+-------------+------+------------+----------------+----------+--------+---+-----------+-------------+



In this case, we found no missing values but in case if we found any, then we have two options either remove the columns or replace the values with some numbers or 0.


1. Method is drop rows with missing values



In [11]:
sales_data= sales_data.dropna()
sales_data.show()

+-------+------+-----------+-------------+------+------------+----------------+----------+--------+-----+-----------+-------------+
|sale_id|branch|       city|customer_type|gender|product_name|product_category|unit_price|quantity|  tax|total_price|reward_points|
+-------+------+-----------+-------------+------+------------+----------------+----------+--------+-----+-----------+-------------+
|      1|     A|   New York|       Member|  Male|     Shampoo|   Personal Care|       5.5|       3| 1.16|      17.66|            1|
|      2|     B|Los Angeles|       Normal|Female|    Notebook|      Stationery|      2.75|      10| 1.93|      29.43|            0|
|      3|     A|   New York|       Member|Female|       Apple|          Fruits|       1.2|      15| 1.26|      19.26|            1|
|      4|     A|    Chicago|       Normal|  Male|   Detergent|       Household|       7.8|       5| 2.73|      41.73|            0|
|      5|     B|Los Angeles|       Member|Female|Orange Juice|       Beverag

2. Method to fill missing values in number columns and strings columns


In [12]:
sales_data = sales_data.fillna({"total_price" : 0})
# replace values in string colms
sales_data= sales_data.fillna({'product_category': 'unknown'})

In [13]:
total_rows = sales_data.count()
total_distincts = sales_data.distinct().count()
print(f"Total rows : {total_rows}, Distinct Columns: {total_distincts}")

Total rows : 1000, Distinct Columns: 1000


After ensuring no duplicates, let’s check the data types to confirm they are as expected (e.g., numbers are numeric, dates are datetime).

In [14]:
sales_data.printSchema()

root
 |-- sale_id: integer (nullable = true)
 |-- branch: string (nullable = true)
 |-- city: string (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_category: string (nullable = false)
 |-- unit_price: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- tax: double (nullable = true)
 |-- total_price: double (nullable = false)
 |-- reward_points: integer (nullable = true)

